# 1️⃣ **Описание шаблона для решения задачи.**

**Задача**: обучить несколько бустингов на 3-х фолдах, выбрать лучшие, усреднить предсказания.

**Модели, которые будем обучать:**
- `CatBoostRegressor`
- `LightGBMRegressor (goss)`
- `XGBoostRegressor (dart)`


✅ Будут выполнены:
- все дополнительные условия
- возможности фреймворков (загрузка датасетов с помощью соответствующих классов, правильная подготовка категориальных признаков, early_stopping, многопоточность)
- подбор гиперпараметров для каждой модели

👀 При желании, рекомендуется проделать следующее:
- Провести EDA (Exploratory Data Analysis) и сделать выводы на основе графики
- Провести Feature Selection
- Провести Object Selection
- Использовать scheduler или custom callbacks
- Обучить дополнительные модели


❗️❗️❗️ **P.S.**
- Данный ноутбук - далеко не единственное верное решение, воспринимайте его как помощник для вашего собственного решения или чтобы побороть страх белого листа :)

- При полном заполнении ноутбука можно получить максимум 9 баллов из 10, так как из дополнительных баллов - только балл за подбор гиперпараметров.

- При любых найденных ошибках/опечатках/непонятных моментов в коде, пишите в [чат курса](https://stepik.org/lesson/681941/step/6?unit=680724)

# 2️⃣ **Подключение необходимых библиотек и загрузка данных.**

In [79]:
!pip install catboost lightgbm==3.2.1 xgboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.2 MB/s eta 0:00:00


In [81]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from catboost import CatBoostRegressor, Pool

import lightgbm as lgb
from lightgbm import Dataset, LGBMRegressor

import xgboost as xgb
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_test.csv')

In [4]:
RANDOM_STATE = 134 # Ваше любимое число :)

In [5]:
results = [] # Здесь будем хранить информацию по каждой модели

# 3️⃣ **Определим вспомогательные функции.**

In [75]:
def train_model(algorithm,
                X,
                y,
                early_stopping_rounds,
                init_params=None,
                cat_features=None,
                random_seed=2023
    ):
    scores = []
    models = []

    kf = KFold(n_splits=3, shuffle=True, random_state=random_seed)

    print(f"========= TRAINING {algorithm.__name__} =========")

    for num_fold, (train_index, val_index) in enumerate(kf.split(X)):
        X_train, X_eval = X.iloc[train_index], X.iloc[val_index]  # Разделение на обучающую и валидационную выборки
        y_train, y_eval = y.iloc[train_index], y.iloc[val_index]

        if init_params is not None:
            model = algorithm(**init_params)
        else:
            model = algorithm()

        if algorithm.__name__ == 'CatBoostRegressor':
            # Используем класс Pool
            train_dataset = Pool(data=X_train, label=y_train, cat_features=cat_features)
            eval_dataset = Pool(data=X_eval, label=y_eval, cat_features=cat_features)

            model.fit(train_dataset,
                      eval_set=eval_dataset,
                      verbose=0,
                      early_stopping_rounds=early_stopping_rounds)

        elif algorithm.__name__ == 'LGBMRegressor':
            # Используем класс Dataset
            train_dataset = Dataset(X_train, label=y_train)
            eval_dataset = Dataset(X_eval, label=y_eval)
            model = lgb.train(params=init_params,
                              train_set=train_dataset,
                              valid_sets=(eval_dataset),
                              categorical_feature=cat_features,
                              callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_rounds)],
                              # early_stopping_rounds=early_stopping_rounds,
                              ) # verbose_eval=False

        elif algorithm.__name__ == 'XGBRegressor':
            # Используем класс DMatrix
            train_dataset = xgb.DMatrix(X_train, label=y_train)
            eval_dataset = xgb.DMatrix(X_eval, label=y_eval)

            model = xgb.train(params=init_params,
                              dtrain=train_dataset,
                              evals=[(train_dataset, 'dtrain'), (eval_dataset, 'dtest')],
                              verbose_eval=False,
                              early_stopping_rounds=early_stopping_rounds)

            X_eval = eval_dataset

        # Сделайте предсказание на X_eval и посчитайте RMSE
        y_pred = model.predict(X_eval)
        score = np.sqrt(mean_squared_error(y_eval, y_pred)) # Подсчет RMSE


        models.append(model)
        scores.append(score)

        print(f'FOLD {num_fold}: SCORE {score}')

    mean_kfold_score = np.mean(scores, dtype="float16") -  np.std(scores, dtype="float16")
    print("\nMEAN RMSE SCORE", mean_kfold_score)

    # Выберите модель с наименьшим значением скора
    best_model = models[np.argmin(scores)]

    return mean_kfold_score, best_model

In [85]:
def tuning_hyperparams(algorithm,
                       X,
                       y,
                       init_params,
                       fit_params,
                       grid_params,
                       n_iter,
                       cv=3,
                       random_state=2023,
    ):
    estimator = algorithm(**init_params)

    # Можно использоавть GridSearchCV
    model = RandomizedSearchCV(estimator=estimator,
                               param_distributions=grid_params,
                               n_iter=n_iter,
                               cv=cv,
                               scoring='neg_root_mean_squared_error',
                               n_jobs=-1,
                               verbose=0,
                               random_state=random_state
    )

    model.fit(X, y, **fit_params)

    return model.best_params_ | init_params

# 4️⃣ **Группируем признаки, отбираем категориальные, выделяем датасет для обучения.**

### EDA

In [13]:
train.head(3)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.1,180.855726,0.023174,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.0,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.1,102.382857,2.513319,174,173


In [11]:
print("### Train ###")
print(train.isna().any())  # Проверка по столбцам
print(train.isna().any().any())  # Проверка по всей таблице

print("### Test ###")
print(test.isna().any())  # Проверка по столбцам
print(test.isna().any().any())  # Проверка по всей таблице

### Train ###
car_id                      False
model                       False
car_type                    False
fuel_type                   False
car_rating                  False
year_to_start               False
riders                      False
year_to_work                False
target_reg                  False
target_class                False
mean_rating                 False
distance_sum                False
rating_min                  False
speed_max                   False
user_ride_quality_median    False
deviation_normal_count      False
user_uniq                   False
dtype: bool
False
### Test ###
car_id                      False
model                       False
car_type                    False
fuel_type                   False
car_rating                  False
year_to_start               False
riders                      False
year_to_work                False
target_class                False
mean_rating                 False
distance_sum                False
rat

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   car_id                    2337 non-null   object 
 1   model                     2337 non-null   object 
 2   car_type                  2337 non-null   object 
 3   fuel_type                 2337 non-null   object 
 4   car_rating                2337 non-null   float64
 5   year_to_start             2337 non-null   int64  
 6   riders                    2337 non-null   int64  
 7   year_to_work              2337 non-null   int64  
 8   target_reg                2337 non-null   float64
 9   target_class              2337 non-null   object 
 10  mean_rating               2337 non-null   float64
 11  distance_sum              2337 non-null   float64
 12  rating_min                2337 non-null   float64
 13  speed_max                 2337 non-null   float64
 14  user_rid

В датасете (И в трейне и в тесте) полностью отсутствуют `nan`, значит не придется их обрабатывать

In [21]:
cat_features = ['car_id', 'model', 'car_type', 'fuel_type']  # категориальные признаки
targets = ['target_reg', 'target_class']  # Целевые переменные
features2drop = ['target_reg', 'target_class']  # Признаки, которые не используются

# Оставляем только нужные признаки
filtered_features = [col for col in train.columns if col not in features2drop]
num_features = ['car_rating', 'year_to_start', 'riders', 'year_to_work', \
                'mean_rating', 'distance_sum', 'rating_min', 'speed_max', \
                'user_ride_quality_median', 'deviation_normal_count', 'user_uniq']  # числовые признаки


print("cat_features", cat_features)
print("num_features", num_features)
print("targets", targets)

cat_features ['car_id', 'model', 'car_type', 'fuel_type']
num_features ['car_rating', 'year_to_start', 'riders', 'year_to_work', 'mean_rating', 'distance_sum', 'rating_min', 'speed_max', 'user_ride_quality_median', 'deviation_normal_count', 'user_uniq']
targets ['target_reg', 'target_class']


In [22]:
X = train[filtered_features].drop(targets, axis=1, errors="ignore")
y = train["target_reg"]

# 5️⃣ **CatBoostRegressor.**



## **Обучение модели.**

In [23]:
cb_init_params = {
    'loss_function': 'RMSE',  # Функция потерь для задачи регрессии
    'eval_metric': 'RMSE',  # Метрика для оценки качества
    'thread_count': -1,  # Использовать все доступные потоки
    'task_type': 'CPU',  # Использование CPU для вычислений
    'random_seed': RANDOM_STATE  # Фиксация случайности
}

cb_score, cb_model = train_model(
    algorithm=CatBoostRegressor,
    X=X, y=y,
    init_params=cb_init_params,
    early_stopping_rounds=50,  # Раннее остановка после 50 итераций без улучшения
    cat_features=cat_features,  # Категориальные признаки
    random_seed=RANDOM_STATE
)

========= TRAINING CatBoostRegressor =========
FOLD 0: SCORE 12.280047125371754
FOLD 1: SCORE 11.281578658938425
FOLD 2: SCORE 11.504727212951305

MEAN RMSE SCORE 11.26


Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [25]:
cb_test_pred = cb_model.predict(test[filtered_features])

pd.DataFrame({'car_id': test['car_id'], 'target_reg': cb_test_pred}).to_csv('cb_pred.csv', index=False)

In [26]:
results.append({
    'model_name': 'CatBoostRegressor',
    'tuning': False,
    'kfold_score': cb_score,
    'leaderboard_score': 12.0,
    'model': cb_model
})

## **Подбор гиперпараметров и обучение модели с новыми параметрами.**

In [28]:
cb_fit_params = {
    'cat_features': cat_features,
    'verbose': 1,
    'early_stopping_rounds': 50  # Установим раннюю остановку на 50 итерациях
}

# Параметры, которые будем перебирать
cb_grid_params = {
    'depth': [4, 6, 8, 10],  # Глубина деревьев
    'learning_rate': [0.01, 0.05, 0.1],  # Скорость обучения
    'l2_leaf_reg': [1, 3, 5, 7],  # Регуляризация
    'iterations': [100, 200, 300]  # Количество итераций
}

catboost_params_after_tuning = tuning_hyperparams(
    algorithm=CatBoostRegressor,
    X=X, y=y,
    init_params=cb_init_params,
    fit_params=cb_fit_params,
    grid_params=cb_grid_params,
    n_iter=20,  # Количество итераций для RandomizedSearchCV
    cv=3,  # 3 фолда для кросс-валидации
    random_state=RANDOM_STATE
)

# Вывод лучших параметров после настройки
catboost_params_after_tuning

0:	learn: 16.9628054	total: 2.42ms	remaining: 240ms
1:	learn: 16.1922609	total: 4.47ms	remaining: 219ms
2:	learn: 15.5496653	total: 6.31ms	remaining: 204ms
3:	learn: 15.1374238	total: 11.2ms	remaining: 269ms
4:	learn: 14.6595388	total: 14.8ms	remaining: 282ms
5:	learn: 14.2837380	total: 16.5ms	remaining: 259ms
6:	learn: 13.9935246	total: 18.8ms	remaining: 249ms
7:	learn: 13.6366796	total: 20.5ms	remaining: 236ms
8:	learn: 13.3620753	total: 22.7ms	remaining: 230ms
9:	learn: 13.1622601	total: 24.8ms	remaining: 223ms
10:	learn: 12.9226101	total: 26.7ms	remaining: 216ms
11:	learn: 12.8117329	total: 28.5ms	remaining: 209ms
12:	learn: 12.6374870	total: 30.3ms	remaining: 203ms
13:	learn: 12.5218453	total: 32.1ms	remaining: 197ms
14:	learn: 12.3863350	total: 34.2ms	remaining: 194ms
15:	learn: 12.2717220	total: 36.2ms	remaining: 190ms
16:	learn: 12.1600840	total: 38.3ms	remaining: 187ms
17:	learn: 12.0576611	total: 40.3ms	remaining: 184ms
18:	learn: 11.9904517	total: 42.2ms	remaining: 180ms
19:

{'learning_rate': 0.1,
 'l2_leaf_reg': 1,
 'iterations': 100,
 'depth': 4,
 'loss_function': 'RMSE',
 'eval_metric': 'RMSE',
 'thread_count': -1,
 'task_type': 'CPU',
 'random_seed': 134}

In [29]:
cb_tuning_score, cb_tuning_model = train_model(algorithm=CatBoostRegressor,
                                               X=X, y=y,
                                               early_stopping_rounds=50,
                                               init_params=catboost_params_after_tuning,
                                               cat_features=cat_features,
                                               random_seed=RANDOM_STATE)

========= TRAINING CatBoostRegressor =========
FOLD 0: SCORE 12.30625493371335
FOLD 1: SCORE 11.226621398344045
FOLD 2: SCORE 11.501449711041762

MEAN RMSE SCORE 11.22


Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [30]:
tuning_cb_test_pred = cb_tuning_model.predict(test[filtered_features])

pd.DataFrame({'car_id': test['car_id'], 'target_reg': tuning_cb_test_pred}).to_csv('tuning_cb_pred.csv', index=False)

In [31]:
results.append({
    'model_name': 'CatBoostRegressor',
    'tuning': True,
    'mean_kfold_score': cb_tuning_score,
    'leaderboard_score': 11.9,
    'model': cb_tuning_model
})

# 6️⃣ **LightGBMRegressor (goss).**

## **Подготовка категориальных признаков.**

[Ссылка](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/4.2%20-%20LightGBM.ipynb), если забыли, как готовить категориальные признаки

In [41]:
le = LabelEncoder()
le_data = pd.concat([train, test])[filtered_features]
le_data

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,4.737759,1.214131e+07,0.10,180.855726,0.023174,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,4.480517,1.803909e+07,0.00,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,4.768391,1.588366e+07,0.10,102.382857,2.513319,174,173
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,3.880920,1.651883e+07,0.10,172.793237,-5.029476,174,170
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,4.181149,1.398317e+07,0.10,203.462289,-14.260456,174,171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,x13640960Q,Nissan Qashqai,standart,petrol,2.44,2015,81070,2016,4.641379,8.035217e+06,0.10,161.000000,5.426149,174,171
1909,Z-2276652N,Renault Kaptur,standart,petrol,4.44,2014,70067,2019,3.956954,1.792881e+07,0.10,195.394955,-6.225923,174,172
1910,F-2165841B,Hyundai Solaris,economy,petrol,5.16,2011,6574,2022,4.607816,1.401163e+07,0.22,113.091551,-4.110241,174,172
1911,x-1331529J,Hyundai Solaris,economy,petrol,5.76,2015,85648,2018,4.462644,1.375324e+07,0.10,154.105593,-4.337686,174,172


In [44]:
X_lgb = X.copy()
test_lgb = test[filtered_features].copy()

# Объединяем train и test для корректной обработки категориальных признаков
le_data = pd.concat([X_lgb, test_lgb], axis=0, ignore_index=True)

le = LabelEncoder()

# Обучаем LabelEncoder на объединённых данных и применяем его
for cat_col in cat_features:
    le.fit(le_data[cat_col].astype(str))
    X_lgb[cat_col] = le.transform(X_lgb[cat_col].astype(str))
    test_lgb[cat_col] = le.transform(test_lgb[cat_col].astype(str))


## **Обучение модели.**

In [64]:
lgb_init_params = {
    'boosting_type': 'goss',  # Используем GOSS boosting
    'n_jobs': -1,  # Задействовать все потоки
    'metric': 'rmse',  # Метрика для оценки
    'objective': 'regression',  # Целевая задача - регрессия
    'random_state': RANDOM_STATE,  # Фиксация случайности
    # 'verbose': -1,  # Подавление лишнего вывода
    'device': 'cpu'  # Использование CPU
}

lgb_score, lgb_model = train_model(
    algorithm=LGBMRegressor,
    X=X_lgb, y=y,
    init_params=lgb_init_params,
    early_stopping_rounds=50,  # Раннее остановка после 50 итераций
    cat_features=cat_features,
    random_seed=RANDOM_STATE,
)

========= TRAINING LGBMRegressor =========
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000531 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1597
[LightGBM] [Info] Number of data points in the train set: 1558, number of used features: 14
[LightGBM] [Info] Using GOSS
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To

Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [65]:
lgb_test_pred = lgb_model.predict(test_lgb)

pd.DataFrame({'car_id': test['car_id'], 'target_reg': lgb_test_pred}).to_csv('lgb_pred.csv', index=False)

In [66]:
results.append({
    'model_name': 'LGBMRegressor (goss)',
    'tuning': False,
    'mean_kfold_score': lgb_score,
    'leaderboard_score': 12.4,
    'model': lgb_model
})

## **Подбор гиперпараметров и обучение модели с новыми параметрами**

In [87]:
lgb_fit_params = {
    'eval_metric': 'rmse',  # Метрика оценки
    'categorical_feature': cat_features,  # Категориальные признаки
    # 'verbose_eval': False,
    'verbose': -1
}

lgb_grid_params = {
    'max_depth': [3, 5, 7, 10],  # Максимальная глубина дерева
    'num_leaves': [15, 31, 63, 127],  # Количество листьев
    'learning_rate': [0.01, 0.05, 0.1],  # Скорость обучения
    'min_data_in_leaf': [10, 20, 30, 50],  # Минимальное число объектов в листе
    'feature_fraction': [0.6, 0.8, 1.0],  # Доля признаков для обучения дерева
    'bagging_fraction': [0.6, 0.8, 1.0],  # Доля выборки для обучения дерева
    'bagging_freq': [0, 5, 10],  # Частота bagging
}



lgb_params_after_tuning = tuning_hyperparams(
    algorithm=LGBMRegressor,
    X=X_lgb,
    y=y,
    init_params=lgb_init_params,
    fit_params=lgb_fit_params,
    grid_params=lgb_grid_params,
    n_iter=50,  # Количество случайных итераций для RandomizedSearchCV
    cv=3,  # Число фолдов в кросс-валидации
    random_state=RANDOM_STATE
)

lgb_params_after_tuning

ValueError: 
All the 150 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py", line 895, in fit
    callbacks=callbacks, init_model=init_model)
  File "/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py", line 595, in fit
  File "/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py", line 549, in get_params
    if isinstance(self, LGBMRegressor):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 248, in get_params
    value = getattr(self, key)
A

In [ ]:
lgb_tuning_score, lgb_tuning_model = train_model(
    algorithm=LGBMRegressor,
    X=X_lgb, y=y,
    init_params=lgb_params_after_tuning,
    early_stopping_rounds=...,
    cat_features=cat_features,
    random_seed=RANDOM_STATE
)

Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [67]:
results.append({
    'model_name': 'LGBMRegressor (goss)',
    'tuning': True,
    'mean_kfold_score': lgb_tuning_score,
    'leaderboard_score': ...,
    'model': lgb_tuning_model
})

# 7️⃣ **XGBoostRegressor (dart).**

## **Подготовка категориальных признаков.**

[Ссылка](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/4.3%20-%20XGBoost.ipynb), если забыли, как готовить категориальные признаки

In [ ]:
X_xgb = X.copy()

# YOUR CODE

## **Обучение модели.**

In [ ]:
xgb_init_params = {
    'enable_categorical': True,
    'booster': ...,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbosity': 0,

    # параметры, которые обязательно объявить, чтобы модель работала в режиме dart
    ...: ...,
    ...: ...,
}


xgb_score, xgb_model = train_model(
    algorithm=XGBRegressor,
    X=X_xgb, y=y,
    init_params=xgb_init_params,
    early_stopping_rounds=...,
    cat_features=cat_features,
    random_seed=RANDOM_STATE
)

Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [ ]:
xgb_test_pred = ... # YOUR CODE

pd.DataFrame({'car_id': test['car_id'], 'target_reg': xgb_test_pred}).to_csv('xgb_pred.csv', index=False)

In [ ]:
results.append({
    'model_name': 'XGBRegressor (dart)',
    'tuning': False,
    'mean_kfold_score': xgb_score,
    'leaderboard_score': ...,
    'model': xgb_model
})

## **Подбор гиперпараметров и обучение модели с новыми параметрами**

In [ ]:
xgb_grid_params = {
    'max_depth': ...,
    'max_leaves': ...,
    # YOUR PARAMS
}


xgb_fit_params = {
    'verbose': False
}


xgb_params_after_tuning = tuning_hyperparams(algorithm=XGBRegressor,
                                             X=X_xgb, y=y,
                                             init_params=xgb_init_params,
                                             fit_params=xgb_fit_params,
                                             grid_params=xgb_grid_params,
                                             n_iter=...,
                                             cv=...,
                                             random_state=RANDOM_STATE
)

xgb_params_after_tuning

In [ ]:
xgb_tuning_score, xgb_tuning_model = train_model(
    algorithm=XGBRegressor,
    X=X_xgb, y=y,
    init_params=xgb_params_after_tuning,
    early_stopping_rounds=...,
    cat_features=cat_features,
    random_seed=RANDOM_STATE
)

In [ ]:
tuning_xgb_test_pred = ... # YOUR CODE

pd.DataFrame({'car_id': test['car_id'], 'target_reg': tuning_xgb_test_pred}).to_csv('tuning_xgb_pred.csv', index=False)

In [ ]:
results.append({
    'model_name': 'XGBRegressor (dart)',
    'tuning': True,
    'mean_kfold_score': xgb_tuning_score,
    'leaderboard_score': ...,
    'model': xgb_tuning_model
})

# 8️⃣ **Финальное предсказание и сохранение лучших моделей**

In [ ]:
best_cb_model = # YOUR CODE
best_cb_model.save_model('best_cb_model.cbm')

best_lgb_model = # YOUR CODE
best_lgb_model.save_model('best_lgb_model.mod')


best_xgb_model = # YOUR CODE
best_xgb_model.save_model('best_xgb_model.json')

In [ ]:
final_cb_pred = # YOUR CODE
final_lgb_pred = # YOUR CODE
final_xgb_pred = # YOUR CODE

final_pred = # YOUR CODE

pd.DataFrame({'car_id': test['car_id'], 'target_reg': final_pred}).to_csv('final_submission.csv', index=False)

# 9️⃣ **Выводы.**


In [ ]:
results = pd.DataFrame(results)
results

Примеры вопросов, на которые можно ответить при формулировании вывода:

- Какая модель показала лучшее качество на валидации/лидерборде?
- Помог ли тюнинг гиперпараметров?
- Помог ли Feature Selection?
- Помог ли Object Selection?
- Что поняли благодаря построенным графикам?
- Улучшилось ли качество на лидерборде после усреднения прогнозов моделей?
- ...

